In [1]:
from osdatahub import FeaturesAPI, Extent, NGD
import geopandas as gpd
import pandas as pd
import folium
import matplotlib.pyplot as plt
import mapclassify as mc
from shapely.geometry import Point
import os
import numpy as np
from convertbng.util import convert_bng, convert_lonlat
from os_functions import OSparam_feature, OSparam_ngd
from folium.plugins import measure_control
from pyaurn import importMeta

key = os.environ.get('OS_API_KEY')
crs = "EPSG:27700"

## Location Setup

In [2]:
AURN_meta = importMeta()
AURN_meta_ongoing = AURN_meta[AURN_meta['end_date'] == 'ongoing']
AURN_meta_ongoing = AURN_meta_ongoing[AURN_meta_ongoing['parameter'] == 'NO']

latlon = AURN_meta_ongoing.longitude[0],AURN_meta_ongoing.latitude[0]
eastnorth = convert_bng(AURN_meta_ongoing.longitude[0],AURN_meta_ongoing.latitude[0])
X = eastnorth[0]
Y = eastnorth[1]
radius = 50
clip = True
product = 'topographic_area'

/tmp/ipykernel_1998/3735478472.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  latlon = AURN_meta_ongoing.longitude[0],AURN_meta_ongoing.latitude[0]
/tmp/ipykernel_1998/3735478472.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  eastnorth = convert_bng(AURN_meta_ongoing.longitude[0],AURN_meta_ongoing.latitude[0])


## Line bearings to Buildings

In [4]:
BuildingData = OSparam_feature(X, Y, radius,product,key, clip)
BuildingGeoSeries = BuildingData['geometry'][BuildingData['Theme'] == 'Buildings']
BuildingGeoSeries = BuildingGeoSeries.to_crs(27700)
BuildingDistances = BuildingGeoSeries.distance(Point(X,Y))
LineDistances = BuildingGeoSeries.shortest_line(Point(X,Y))

In [5]:
import folium
from shapely.geometry import Point, LineString

# Define the line and start/end points
line = LineDistances.iloc[0]
end_point = Point(line.coords[0])
start_point = Point(line.coords[-1])
line_points = [p for p in line.coords]
lats = convert_lonlat(*zip(*line_points))[0]
lons = convert_lonlat(*zip(*line_points))[1]

points = []
for i in range(len(lats)):
    points.append([lons[i], lats[i]])

start_lon, start_lat = convert_lonlat(start_point.x, start_point.y)
end_lon,end_lat = convert_lonlat(end_point.x, end_point.y)

# Define the projection
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')
fwd_azimuth,back_azimuth,distance = geodesic.inv(start_lon[0], start_lat[0], end_lon[0], end_lat[0])

# Create a folium map centered on the line
m = folium.Map(location=[start_lat[0],start_lon[0]], zoom_start=19)

# # Add the line to the map in blue
folium.PolyLine(points, color='blue',popup=fwd_azimuth).add_to(m)

# # Add the start point to the map in red
folium.Marker(location=[start_lat[0],start_lon[0]], icon=folium.Icon(color='green')).add_to(m)

# # Add the end point to the map in green
folium.Marker(location=[end_lat[0], end_lon[0]], icon=folium.Icon(color='red')).add_to(m)

# # Display the map
m
# points

## NGD

In [11]:
ngd_collections = NGD.get_collections()

In [12]:
product = 'trn-rami-averageandindicativespeed-1'
extent = Extent.from_radius((X,Y), 50, "EPSG:27700")

In [13]:
collections = {}

for i in ngd_collections['collections']:
    collections.update({i['title']:i['id']})

recipe = ['Average And Indicative Speed',
'Pavement Link',
'Pavement',
'Road Link',]

In [14]:
import difflib

data_collections = [collections[difflib.get_close_matches(i, collections.keys())[0]] for i in recipe]
data_collections


['trn-rami-averageandindicativespeed-1',
 'trn-ntwk-pavementlink-1',
 'trn-ntwk-pavementlink-1',
 'trn-ntwk-roadlink-2']

In [12]:
buildings = OSparam_feature(X, Y, radius,'topographic_area',key, clip)
buildings = buildings[buildings['Theme'] == 'Buildings'].set_crs(27700)
averagespeed = OSparam_ngd(X, Y, radius,data_collections[0],key, clip=False)
pavement = OSparam_ngd(X, Y, radius,data_collections[1],key, clip=False)
road = OSparam_ngd(X, Y, radius,data_collections[2],key, clip=False)

In [13]:
buildings_headers = ['AbsHMin','AbsH2', 'AbsHMax', 'RelH2', 'RelHMax', 'BHAConf', 'Shape_Length','Shape_Area']
averagespeed_headers = list(averagespeed.columns)[6:-1]
pavement_headers = list(pavement.columns)[6:-1]
road_headers = list(road.columns)[6:-1]

In [14]:
m = folium.Map(location=[latlon[1], latlon[0]], zoom_start=18)
fg = folium.FeatureGroup(name="data")
fg.add_child(folium.Marker([latlon[1], latlon[0]]))
fg.add_child(folium.GeoJson(buildings,popup=folium.GeoJsonPopup(fields=buildings_headers),style_function=lambda x: {'color':'black'}))
fg.add_child(folium.GeoJson(averagespeed.set_crs(4326),popup=folium.GeoJsonPopup(fields=averagespeed_headers),style_function=lambda x: {'color':'red'}))
fg.add_child(folium.GeoJson(pavement.set_crs(4326),popup=folium.GeoJsonPopup(fields=pavement_headers),style_function=lambda x: {'color':'green'}))
fg.add_child(folium.GeoJson(road.set_crs(4326),popup=folium.GeoJsonPopup(fields=road_headers),style_function=lambda x: {'color':'blue'}))

m.add_child(fg)

m



## Features

In [11]:
from osdatahub.FeaturesAPI.feature_products import PREMIUM_NEW,OPEN_NEW
PREMIUM_NEW,OPEN_NEW

({'topographic_area': Product(name='Topography_TopographicArea', geometry='MultiPolygon'),
  'topographic_point': Product(name='Topography_TopographicPoint', geometry='Point'),
  'topographic_line': Product(name='Topography_TopographicLine', geometry='MultiLineString'),
  'water_network_link': Product(name='WaterNetwork_WatercourseLink', geometry='MultiLineString'),
  'water_network_node': Product(name='WaterNetwork_HydroNode', geometry='Point'),
  'path_network_link': Product(name='DetailedPathNetwork_RouteLink', geometry='MultiLineString'),
  'path_network_node': Product(name='DetailedPathNetwork_RouteNode', geometry='Point'),
  'highways_connecting_link': Product(name='Highways_ConnectingLink', geometry='MultiLineString'),
  'highways_connecting_node': Product(name='Highways_ConnectingNode', geometry='Point'),
  'highways_ferry_link': Product(name='Highways_FerryLink', geometry='MultiLineString'),
  'highways_ferry_node': Product(name='Highways_FerryNode', geometry='Point'),
  'high

## EDA

Using the AURN stations as our sample, we'll explore the OS data hub API data, broken down into the 4 physical baseline characteristics we're interested in, before then exploring the data transformations.

### Buildings

In [3]:
AURN_meta_ongoing

,site_id,site_name,location_type,latitude,longitude,parameter,Parameter_name,start_date,end_date,ratified_to,zone,agglomeration,local_authority
rownames,,,,,,,,,,,,,
25,ABD7,Aberdeen Union Street Roadside,Urban Traffic,57.144555,-2.106472,NO,Nitric oxide,2008-01-01,ongoing,2023-09-30,North East Scotland,NaN,Aberdeen City
31,ABD8,Aberdeen Wellington Road,Urban Traffic,57.133888,-2.094198,NO,Nitric oxide,2016-02-09,ongoing,2023-09-30,North East Scotland,NaN,Aberdeen City
37,ARM6,Armagh Roadside,Urban Traffic,54.353728,-6.654558,NO,Nitric oxide,2009-01-01,ongoing,2023-09-30,Northern Ireland,NaN,"Armagh, Banbridge and Craigavon"
94,BAAR,Ballymena Antrim Road,Urban Traffic,54.851491,-6.274961,NO,Nitric oxide,2017-04-01,ongoing,2023-09-30,Northern Ireland,NaN,Mid and East Antrim
100,BALM,Ballymena Ballykeel,Urban Background,54.861595,-6.250873,NO,Nitric oxide,2016-05-01,ongoing,2023-09-30,Northern Ireland,NaN,Mid and East Antrim
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,WAR,Warrington,Urban Background,53.389225,-2.615593,NO,Nitric oxide,2008-10-21,ongoing,2023-06-30,North West & Merseyside,NaN,Warrington
2705,WSMR,Widnes Milton Road,Urban Traffic,53.365391,-2.731680,NO,Nitric oxide,2015-03-09,ongoing,2023-09-30,North West & Merseyside,NaN,Halton
2755,WTHG,Worthing A27 Roadside,Urban Traffic,50.832947,-0.379916,NO,Nitric oxide,2016-10-01,ongoing,2023-09-30,South East,Brighton/Worthing/Littlehampton,Worthing


In [5]:
AURN_buildings = gpd.GeoDataFrame()
line_distances = gpd.GeoSeries()
for index, row in AURN_meta_ongoing.iterrows():
    eastnorth = convert_bng(row['longitude'],row['latitude'])
    buildings = OSparam_feature(eastnorth[0], eastnorth[1], radius,'topographic_area',key, clip=False)
    if isinstance(buildings, str):

        continue
    else:
        buildings_data = buildings[buildings['Theme'] == 'Buildings'].set_crs(27700)
        LineDistances = buildings_data.shortest_line(Point(eastnorth[0], eastnorth[1]))
        ## ''' need to add in bearings ''' ##
        buildings_data['distance_to_point'] = LineDistances.length
        AURN_buildings = pd.concat([AURN_buildings,buildings_data])
        line_distances = pd.concat([line_distances,LineDistances])
        AURN_buildings.to_csv('AURN_buildings.csv')


/tmp/ipykernel_1998/2785648013.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  AURN_buildings = pd.concat([AURN_buildings,buildings_data])
/tmp/ipykernel_1998/2785648013.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  AURN_buildings = pd.concat([AURN_buildings,buildings_data])


In [3]:
AURN_buildings_headers = ['AbsHMin','AbsH2', 'AbsHMax', 'RelH2', 'RelHMax', 'BHAConf', 'Shape_Length','Shape_Area','distance_to_point']

In [6]:
m = folium.Map(location=[51.861595,-1.250873], zoom_start=5, tiles='cartodbpositron')
fg = folium.FeatureGroup(name="data")
AURN_meta_ongoing.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]], 
                                              radius=2, popup=row['site_name'])
                                             .add_to(m), axis=1)
fg.add_child(folium.GeoJson(AURN_buildings,popup=folium.GeoJsonPopup(fields=AURN_buildings_headers),style_function=lambda x: {'color':'black'}))
fg.add_child(folium.GeoJson(line_distances,style_function=lambda x: {'color': 'red', 'weight': 1, 'opacity':1}))
m.add_child(fg)

m


In [ ]:
AURN_buildings.head()

### Roads

In [19]:
AURN_roads = gpd.GeoDataFrame()
for index, row in AURN_meta_ongoing.iterrows():
    eastnorth = convert_bng(row['longitude'],row['latitude'])
    road = OSparam_ngd(eastnorth[0], eastnorth[1], radius,data_collections[2],key, clip=False)
    if isinstance(road, str):
        continue
    else:
        AURN_roads = pd.concat([AURN_roads,road])
        AURN_roads.to_csv('AURN_roads.csv')

No features found
No features found
No features found
No features found
No features found
No features found
No features found
No features found
No features found
No features found


In [20]:
AURN_roads.head()

,geometry,osid,theme,changetype,name1_text,name2_text,roadlinkid,description,versiondate,name1_language,...,alternatename2_language,roadclassificationnumber,versionavailablefromdate,presenceofpavement_source,presenceofpavement_sideofroad,presenceofpavement_updatedate,presenceofpavement_evidencedate,presenceofpavement_capturemethod,presenceofpavement_averagewidth_m,presenceofpavement_minimumwidth_m
0,"LINESTRING (-2.10849 57.14385, -2.10679 57.14433)",564e4f44-1e02-4cd7-bf20-4ccc3ddcb217,Transport,Modified Attributes,Union Street,None,c7747d96-439d-4961-ae07-0fde70a3476e,Pavement,2023-11-10,None,...,None,None,2023-11-11T00:00:00Z,Ordnance Survey,Left,2023-11-07,2014-11-09,Automated Process,3.40,0.53
1,"LINESTRING (-2.10693 57.14452, -2.10682 57.14440)",5bc620db-6178-4a24-9c2e-78df456f8f0f,Transport,Modified Attributes,Union Row,None,4ca68012-3871-4046-b22d-66f53c42ef03,Pavement,2023-11-10,None,...,None,None,2023-11-11T00:00:00Z,Ordnance Survey,Right,2023-11-07,2014-11-09,Automated Process,2.27,1.11
2,"LINESTRING (-2.10669 57.14435, -2.10564 57.14464)",618c14cb-a5e8-4402-9d18-185c629afc0b,Transport,Modified Attributes,Union Street,None,dc5f9bbb-9dfe-40f2-8cfe-630784a0bc30,Pavement,2023-11-10,None,...,None,None,2023-11-11T00:00:00Z,Ordnance Survey,Left,2023-11-07,2014-11-09,Automated Process,4.23,1.00
3,"LINESTRING (-2.10721 57.14480, -2.10719 57.144...",62c3e9db-d0a8-45c3-b0ad-e4975c964588,Transport,Modified Geometry And Attributes,Union Row,None,4ca68012-3871-4046-b22d-66f53c42ef03,Pavement,2023-11-10,None,...,None,None,2023-11-11T00:00:00Z,Ordnance Survey,Right,2023-11-07,2014-11-09,Automated Process,2.27,1.11
4,"LINESTRING (-2.10670 57.14427, -2.10629 57.143...",826b9208-3570-4025-ac1b-14f0c5e6ebbc,Transport,Modified Attributes,Bon-Accord Street,None,3c2a54ab-f0ee-437d-b07f-6e947357e845,Pavement,2023-11-10,None,...,None,None,2023-11-11T00:00:00Z,Ordnance Survey,Left,2023-11-07,2019-08-25,Automated Process,2.23,0.89
